<a href="https://colab.research.google.com/github/VOTUANANH01/DS104_Distributed_and_parallel_computing/blob/main/GraphFrame_Ex2%2BEx3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/VOTUANANH01/DS104_Distributed_and_parallel_computing

Cloning into 'DS104_Distributed_and_parallel_computing'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 11), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [ ]:
!pip install pyspark==3.0.1
%cd /usr/local/lib/python3.7/dist-packages/pyspark/jars
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
%cd '/content/'
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
!jar xf /content/graphframes-0.8.1-spark3.0-s_2.12.jar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=7c0ae502dde1eba3a74d325f5b449ffb7daa51578732d7d1b07fee87a47e2049
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
/usr/local/lib/python3.7/dist-packages/pyspark/jars
--2022-09-27 01:56:42--  https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 18.165.83.74, 18.165.83.27, 18.165.83.55, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|18.165.83.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248133 (242K) [binary/octet-strea

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark=SparkSession.builder.getOrCreate()

#Example 2: Graph and Bike Trip Data

In [ ]:
#read data
station =spark.read.option("header",True).csv("/content/DS104_Distributed_and_parallel_computing/Slide 5-6 data/station.csv")
trip =spark.read.option("header",True).csv("/content/DS104_Distributed_and_parallel_computing/Slide 5-6 data/trip.csv")


In [ ]:
station.show()
trip.show()

+---+--------------------+------------------+-------------------+----------+------------+-----------------+
| id|                name|               lat|               long|dock_count|        city|installation_date|
+---+--------------------+------------------+-------------------+----------+------------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|    San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|    San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|    San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|    San Jose|         8/5/2013|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|    San Jose|         8/7/2013|
|  7|Paseo de San Antonio|         37.333798|-121.88694299999999|        15|    San Jose|         8/7/2013|
|  8| San Salvador at 1st|  

In [ ]:
station.printSchema()
trip.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- dock_count: string (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [ ]:
tripEdges=trip.withColumnRenamed("start_station_name","src")\
  .withColumnRenamed("end_station_name","dst")\
  .withColumn("start_date",unix_timestamp('start_date','yyyy-MM-dd').cast("date"))
tripEdges.show(5)

AnalysisException: ignored

In [ ]:
from graphframes import *
stationGraph=GraphFrame(station,tripEdges)

##Trip from Station to Station

In [ ]:
#count number of trips that are exact same from station A to station B
from pyspark.sql.functions import *
topTrips=stationGraph.edges.groupBy("src","dst").count()\
  .orderBy(desc("count")).limit(10)


In [ ]:
topTrips.show(truncate=False)

+----------------------------------------+----------------------------------------+-----+
|src                                     |dst                                     |count|
+----------------------------------------+----------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend) |Townsend at 7th                         |6216 |
|Harry Bridges Plaza (Ferry Building)    |Embarcadero at Sansome                  |6164 |
|Townsend at 7th                         |San Francisco Caltrain (Townsend at 4th)|5041 |
|2nd at Townsend                         |Harry Bridges Plaza (Ferry Building)    |4839 |
|Harry Bridges Plaza (Ferry Building)    |2nd at Townsend                         |4357 |
|Embarcadero at Sansome                  |Steuart at Market                       |4269 |
|Embarcadero at Folsom                   |San Francisco Caltrain (Townsend at 4th)|3967 |
|Steuart at Market                       |2nd at Townsend                         |3903 |
|2nd at So

##In Degrees and Out Degrees

In [ ]:
#in-degree: the number of edges coming into vertex in directed graph
inDeg=stationGraph.inDegrees
inDeg.orderBy(desc("inDegree")).show(10,truncate=False)


+---------------------------------------------+--------+
|id                                           |inDegree|
+---------------------------------------------+--------+
|San Francisco Caltrain (Townsend at 4th)     |63179   |
|San Francisco Caltrain 2 (330 Townsend)      |35117   |
|Harry Bridges Plaza (Ferry Building)         |33193   |
|Embarcadero at Sansome                       |30796   |
|2nd at Townsend                              |28529   |
|Market at Sansome                            |28033   |
|Townsend at 7th                              |26637   |
|Steuart at Market                            |25025   |
|Temporary Transbay Terminal (Howard at Beale)|23080   |
|Market at 4th                                |19915   |
+---------------------------------------------+--------+
only showing top 10 rows



In [ ]:
#out-degree: the number of edges going out of vertex in directed vertex in directed graph
outDeg=stationGraph.outDegrees
outDeg.orderBy(desc("outDegree")).show(10,truncate=False)

+---------------------------------------------+---------+
|id                                           |outDegree|
+---------------------------------------------+---------+
|San Francisco Caltrain (Townsend at 4th)     |49092    |
|San Francisco Caltrain 2 (330 Townsend)      |33742    |
|Harry Bridges Plaza (Ferry Building)         |32934    |
|Embarcadero at Sansome                       |27713    |
|Temporary Transbay Terminal (Howard at Beale)|26089    |
|2nd at Townsend                              |25837    |
|Steuart at Market                            |24838    |
|Market at Sansome                            |24172    |
|Townsend at 7th                              |23724    |
|Market at 10th                               |20272    |
+---------------------------------------------+---------+
only showing top 10 rows



In [ ]:
#which station with highest ratio of in-degree but fewest out-degree
#meaning: station where trips end but rarely start from
degreeRatio=inDeg.join(outDeg,inDeg.id==outDeg.id).drop(outDeg.id)\
  .selectExpr("id","double(inDegree)/double(outDegree) as degreeRatio")

degreeRatio.orderBy(desc("degreeRatio")).show(10,truncate=False)

+----------------------------------------+------------------+
|id                                      |degreeRatio       |
+----------------------------------------+------------------+
|Redwood City Medical Center             |1.4533762057877813|
|Redwood City Public Library             |1.300469483568075 |
|San Francisco Caltrain (Townsend at 4th)|1.286951030717836 |
|Washington at Kearny                    |1.2723671947809878|
|MLK Library                             |1.233038348082596 |
|SJSU 4th at San Carlos                  |1.2282051282051283|
|San Mateo County Center                 |1.2195121951219512|
|Broadway at Main                        |1.208955223880597 |
|University and Emerson                  |1.2056878306878307|
|Washington at Kearney                   |1.203804347826087 |
+----------------------------------------+------------------+
only showing top 10 rows



In [ ]:
#trips start from the statio but don't end  there often
degreeRatio.orderBy(asc("degreeRatio")).show(10,truncate=False)

+-------------------------------+------------------+
|id                             |degreeRatio       |
+-------------------------------+------------------+
|Grant Avenue at Columbus Avenue|0.564700110388814 |
|2nd at Folsom                  |0.6056461731493099|
|Powell at Post (Union Square)  |0.6887003841229193|
|San Jose City Hall             |0.6928541579607188|
|San Francisco City Hall        |0.7497243660418964|
|Beale at Market                |0.774906104780699 |
|Redwood City Caltrain Station  |0.8075933075933076|
|Golden Gate at Polk            |0.8153091800599291|
|Evelyn Park and Ride           |0.8218356328734253|
|Ryland Park                    |0.8248425872925015|
+-------------------------------+------------------+
only showing top 10 rows



#Example 3: Flight Data

In [ ]:
flightDF=spark.read.json('/content/DS104_Distributed_and_parallel_computing/Slide 5-6 data/flightdata2018.json')


In [ ]:
flightDF.head()

Row(arrdelay=0.0, carrier='DL', crsarrtime=1116, crsdephour=9, crsdeptime=850, crselapsedtime=146.0, depdelay=0.0, dist=946.0, dofW=1, dst='BOS', fldate='2018-01-01', id='ATL_BOS_2018-01-01_DL_104', month=1, src='ATL')

In [ ]:
flightEdges=flightDF.withColumnRenamed("depdelay","delay")\
  .select("id","fldate","month","dofW","carrier","src","dst","crsdephour","crsdeptime"\
  ,"delay","crsarrtime","arrdelay","crselapsedtime","dist")

In [ ]:
flightEdges.show()

+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+-----+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|delay|crsarrtime|arrdelay|crselapsedtime| dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+-----+
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|         9|       850|  0.0|      1116|     0.0|         146.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        11|      1122|  8.0|      1349|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        14|      1356|  9.0|      1623|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        16|      1620|  0.0|      1851|     3.0|         151.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        19|      1940|  6.0|  

In [ ]:
flightEdges.printSchema()

root
 |-- id: string (nullable = true)
 |-- fldate: string (nullable = true)
 |-- month: long (nullable = true)
 |-- dofW: long (nullable = true)
 |-- carrier: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- crsdephour: long (nullable = true)
 |-- crsdeptime: long (nullable = true)
 |-- delay: double (nullable = true)
 |-- crsarrtime: long (nullable = true)
 |-- arrdelay: double (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- dist: double (nullable = true)



In [ ]:
flightEdges.count()

282628

In [ ]:
airportVertices=spark.read.json("/content/DS104_Distributed_and_parallel_computing/Slide 5-6 data/airports.json")

In [ ]:
airportVertices.show()

+-------------+-------+-----+---+
|         City|Country|State| id|
+-------------+-------+-----+---+
|      Chicago|    USA|   IL|ORD|
|     New York|    USA|   NY|LGA|
|       Boston|    USA|   MA|BOS|
|      Houston|    USA|   TX|IAH|
|       Newark|    USA|   NJ|EWR|
|       Denver|    USA|   CO|DEN|
|        Miami|    USA|   FL|MIA|
|San Francisco|    USA|   CA|SFO|
|      Atlanta|    USA|   GA|ATL|
|       Dallas|    USA|   TX|DFW|
|    Charlotte|    USA|   NC|CLT|
|  Los Angeles|    USA|   CA|LAX|
|      Seattle|    USA|   WA|SEA|
+-------------+-------+-----+---+



##Create graphframe

In [ ]:
from graphframes import *
flightGraph=GraphFrame(airportVertices,flightEdges)
print(flightGraph)


GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 12 more fields])


##Querying the GraphFrame

In [ ]:
#show vertices amd edges
flightGraph.vertices.show(5)
flightGraph.edges.show(5)


+--------+-------+-----+---+
|    City|Country|State| id|
+--------+-------+-----+---+
| Chicago|    USA|   IL|ORD|
|New York|    USA|   NY|LGA|
|  Boston|    USA|   MA|BOS|
| Houston|    USA|   TX|IAH|
|  Newark|    USA|   NJ|EWR|
+--------+-------+-----+---+
only showing top 5 rows

+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+-----+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|delay|crsarrtime|arrdelay|crselapsedtime| dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+-----+
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|         9|       850|  0.0|      1116|     0.0|         146.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        11|      1122|  8.0|      1349|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|    

In [ ]:
#which flight routes have longest distance?

from pyspark.sql.functions import *
flightGraph.edges.groupBy("src","dst").max("dist").sort(desc("max(dist)")).show(5)

+---+---+---------+
|src|dst|max(dist)|
+---+---+---------+
|MIA|SEA|   2724.0|
|SEA|MIA|   2724.0|
|BOS|SFO|   2704.0|
|SFO|BOS|   2704.0|
|LAX|BOS|   2611.0|
+---+---+---------+
only showing top 5 rows



In [ ]:
#count of departure delays by Origin and destination
flightGraph.edges.filter("delay > 40").groupBy("src","dst")\
  .agg(count("delay").alias("flightcount")).sort(desc("flightcount")).show(5)

+---+---+-----------+
|src|dst|flightcount|
+---+---+-----------+
|ORD|LGA|        588|
|LAX|SFO|        578|
|ATL|EWR|        561|
|LGA|ORD|        532|
|ORD|SFO|        470|
+---+---+-----------+
only showing top 5 rows



In [ ]:
#what are the longest delay for flights that are greater than 100 milesin distance?

flightGraph.edges.filter("dist > 1500").orderBy(desc("delay")).show(5)

+--------------------+----------+-----+----+-------+---+---+----------+----------+------+----------+--------+--------------+------+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime| delay|crsarrtime|arrdelay|crselapsedtime|  dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+------+----------+--------+--------------+------+
|BOS_DFW_2018-06-2...|2018-06-28|    6|   4|     AA|BOS|DFW|        10|      1012|1345.0|      1326|  1332.0|         254.0|1562.0|
|LAX_MIA_2018-07-0...|2018-07-09|    7|   1|     AA|LAX|MIA|         9|       945|1283.0|      1811|  1373.0|         326.0|2342.0|
|BOS_LAX_2018-03-2...|2018-03-28|    3|   3|     AA|BOS|LAX|        10|      1040|1242.0|      1427|  1234.0|         407.0|2611.0|
|DEN_EWR_2018-07-2...|2018-07-20|    7|   5|     UA|DEN|EWR|        11|      1117|1125.0|      1713|  1106.0|         236.0|1605.0|
|SEA_DFW_2018-07-1...|2018-07-10|    7|   2|     AA|SEA|DFW|        15|     

In [ ]:
#what destination states tend to have significant delays departing from SEA?
#state with longest cummulative delays (with individual delays > 100 minutes) (origin: Seattle)

flightGraph.edges.filter("src = 'SEA' and delay > 100").show(10)

+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+------+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|delay|crsarrtime|arrdelay|crselapsedtime|  dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+------+
|SEA_SFO_2018-01-0...|2018-01-01|    1|   1|     UA|SEA|SFO|         5|       510|281.0|       723|   281.0|         133.0| 679.0|
|SEA_DEN_2018-01-0...|2018-01-02|    1|   2|     WN|SEA|DEN|        13|      1335|130.0|      1715|   130.0|         160.0|1024.0|
|SEA_DEN_2018-01-0...|2018-01-02|    1|   2|     WN|SEA|DEN|         8|       840|369.0|      1220|   347.0|         160.0|1024.0|
|SEA_LAX_2018-01-0...|2018-01-02|    1|   2|     DL|SEA|LAX|        19|      1942|190.0|      2220|   186.0|         158.0| 954.0|
|SEA_MIA_2018-01-0...|2018-01-02|    1|   2|     AA|SEA|MIA|        13|      1307|1

In [ ]:
#what is the average delay for delay flights departing from Boston?
flightGraph.edges.filter("src = 'BOS' and delay > 1")\
.groupBy("src","dst").avg("delay").sort(desc("avg(delay)")).show(5)

+---+---+------------------+
|src|dst|        avg(delay)|
+---+---+------------------+
|BOS|EWR|57.078947368421055|
|BOS|LGA| 54.00142857142857|
|BOS|ORD|          47.97375|
|BOS|SEA| 45.56565656565657|
|BOS|DFW| 43.30031948881789|
+---+---+------------------+
only showing top 5 rows



In [ ]:
#which airport has the most incoming flights? the most outgoing?
flightGraph.inDegrees.orderBy(desc("inDegree")).show(5)
flightGraph.outDegrees.orderBy(desc("outDegree")).show(5)

+---+--------+
| id|inDegree|
+---+--------+
|ORD|   32314|
|ATL|   30217|
|LAX|   26869|
|DFW|   24947|
|DEN|   23202|
+---+--------+
only showing top 5 rows

+---+---------+
| id|outDegree|
+---+---------+
|ORD|    32072|
|ATL|    30165|
|LAX|    26864|
|DFW|    24972|
|DEN|    23410|
+---+---------+
only showing top 5 rows



In [ ]:
#what are the highest degree vertices (most incoming and outgoing flights)
flightGraph.degrees.orderBy(desc("degree")).show(3)

+---+------+
| id|degree|
+---+------+
|ORD| 64386|
|ATL| 60382|
|LAX| 53733|
+---+------+
only showing top 3 rows



In [ ]:
#what are the 3 most frequent flight in the dataset?

flightGraph.edges.groupBy("src","dst").count().orderBy(desc("count")).show(3)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|LGA|ORD| 4442|
|ORD|LGA| 4426|
|LAX|SFO| 4406|
+---+---+-----+
only showing top 3 rows



In [ ]:
#what delyas might we blame on SFO?
#With SFO as the connecting city (b), we are looking for all flights 
#[ab] from any origin city (a) that will connect to SFO (b) 
#prior to flying [bc] to any destination city (c). 
#We are filtering it such that the delay 
#for either flight ([ab] or [bc]) is greater than 500 minutes 
#and the second flight (bc) occurred within 
#approximately a day of the first flight (ab).
motif= flightGraph.find("(a)-[ab]->(b);(b)-[bc]->(c)")\
  .filter("(b.id ='SFO') and (ab.delay > 500 or bc.delay > 500) ") \
  .selectExpr("a.id as a","ab.delay as ab","b.id as b","bc.delay as bc","c.id as c")
motif.show(n=5)

+---+----+---+-----+---+
|  a|  ab|  b|   bc|  c|
+---+----+---+-----+---+
|ATL|51.0|SFO|545.0|ORD|
|ATL|51.0|SFO|585.0|BOS|
|ATL|51.0|SFO|631.0|BOS|
|ATL|51.0|SFO|552.0|BOS|
|ATL|51.0|SFO|568.0|ORD|
+---+----+---+-----+---+
only showing top 5 rows



In [ ]:
#determining airport ranking of importance using PageRank

ranks=flightGraph.pageRank(resetProbability = 0.15,maxIter=5)

ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).show(20)

+-------------+-------+-----+---+------------------+
|         City|Country|State| id|          pagerank|
+-------------+-------+-----+---+------------------+
|      Chicago|    USA|   IL|ORD|1.4212550221589475|
|      Atlanta|    USA|   GA|ATL| 1.339101482426853|
|  Los Angeles|    USA|   CA|LAX| 1.200976131283591|
|       Dallas|    USA|   TX|DFW|1.1270531678567586|
|       Denver|    USA|   CO|DEN|1.0591136061016508|
|San Francisco|    USA|   CA|SFO|1.0246282632093013|
|     New York|    USA|   NY|LGA|0.9447562478927347|
|       Boston|    USA|   MA|BOS|0.8775066074875163|
|       Newark|    USA|   NJ|EWR|0.8731036205114817|
|        Miami|    USA|   FL|MIA|0.8507878300790072|
|      Houston|    USA|   TX|IAH|0.8350718288420121|
|    Charlotte|    USA|   NC|CLT|0.8049000935422239|
|      Seattle|    USA|   WA|SEA|0.6417460986079204|
+-------------+-------+-----+---+------------------+



In [ ]:
flightGraph.vertices.select("id").show()

+---+
| id|
+---+
|ORD|
|LGA|
|BOS|
|IAH|
|EWR|
|DEN|
|MIA|
|SFO|
|ATL|
|DFW|
|CLT|
|LAX|
|SEA|
+---+



In [ ]:
filteredPaths = flightGraph.bfs( \
fromExpr = "id = 'SFO'",\
toExpr = "id = 'LGA'", \
maxPathLength = 3)
filteredPaths.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[San Francisco, U...|[SFO_IAH_2018-01-...|[Houston, USA, TX...|[IAH_LGA_2018-01-...|[New York, USA, N...|
|[San Francisco, U...|[SFO_IAH_2018-01-...|[Houston, USA, TX...|[IAH_LGA_2018-01-...|[New York, USA, N...|
|[San Francisco, U...|[SFO_IAH_2018-01-...|[Houston, USA, TX...|[IAH_LGA_2018-01-...|[New York, USA, N...|
|[San Francisco, U...|[SFO_IAH_2018-01-...|[Houston, USA, TX...|[IAH_LGA_2018-01-...|[New York, USA, N...|
|[San Francisco, U...|[SFO_IAH_2018-01-...|[Houston, USA, TX...|[IAH_LGA_2018-01-...|[New York, USA, N...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
filteredPaths = flightGraph.bfs( \
fromExpr = "id = 'SFO'",\
toExpr = "id = 'LGA'", \
maxPathLength = 2)
filteredPaths.show(5)

+----+-------+-----+---+
|City|Country|State| id|
+----+-------+-----+---+
+----+-------+-----+---+

